---
We'll use tensorflow to predict the type of shape in each image.

We should already have `.npy` files in `greyscale-data`.  We'll first load data into various structures for later.  This cell mainly splits the data into training, validation and test folds.  To make the splits I'm hashing filenames and then sorting those hashes alphabetically -- this mixes up the images, but makes the mixing deterministic.

---

In [ ]:
import hashlib


input_directory = 'greyscaled-data'

# Load all the data into an array.
# Each element is a tuple: (filename, numpy data).
# The filename structure is "<number>-<color>-<texture>-<shape>-<rotation>.png"
all_data = [
  (f, np.load(os.path.join(input_directory, f))) for f in os.listdir(input_directory)
]

# Hash the filename and sort the hashes alphabetically.
all_data_with_hashes = [
  (filename, hashlib.md5(filename).hexdigest(), data) for filename, data in all_data
]
all_data_sorted = sorted(all_data_with_hashes, key=lambda element: element[1])

# Save 20% of the data for testing (the final, one-shot evaluation of performance).
split_index = int(0.2 * len(all_data_sorted))
test_data = all_data_sorted[0:split_index]
remaining_data = all_data_sorted[split_index:]

# Now save 20% of the remaining data for validation.
split_index = int(0.2 * len(remaining_data))
validation_data = remaining_data[0:split_index]
training_data = remaining_data[split_index:]

# For convenience, get all the pixel data into separate arrays.
training_pixel_data = [pixel_data for _, _, pixel_data in training_data]
validation_pixel_data = np.array([pixel_data for _, _, pixel_data in validation_data])
test_pixel_data = np.array([pixel_data for _, _, pixel_data in test_data])

# Each filename, in its text, has an embedded type of shape.
# As in, "2-red-empty-oval-45.npy"
# We need to convert those classes (the output ground truth) into label arrays.
all_labels = {
  'oval': [1., 0., 0.],
  'diamond': [0., 1., 0.],
  'bean': [0., 0., 1.],
}
training_labels = [
  all_labels[filename.split('-')[3]] for filename, _, _ in training_data
]
validation_labels = [
  all_labels[filename.split('-')[3]] for filename, _, _ in validation_data
]
test_labels = [
  all_labels[filename.split('-')[3]] for filename, _, _ in test_data
]

---
setup tensorflow

---

In [ ]:
import random
import tensorflow as tf


learning_rate = 0.0000005
regularization_factor = 1e-4
card_width, card_height = 150, 150
first_hidden_layer_size, second_hidden_layer_size = 1024, 256

graph = tf.Graph()
with graph.as_default():
  # Setup the training steps.
  tf_training_data = tf.placeholder(tf.float32, shape=[None, card_width*card_height*card_channels])
  tf_training_labels = tf.placeholder(tf.float32, shape=[None, 3])
  
  # Create hidden layers of ReLUs.
  first_hidden_weights = tf.Variable(
    tf.truncated_normal([card_width*card_height*card_channels, first_hidden_layer_size]), name='first_hidden_weights')
  first_hidden_biases = tf.Variable(
    tf.zeros([first_hidden_layer_size]), name='first_hidden_biases')
  first_hidden_layer = tf.nn.relu(tf.matmul(tf_training_data, first_hidden_weights) + first_hidden_biases)
  second_hidden_weights = tf.Variable(
    tf.truncated_normal([first_hidden_layer_size, second_hidden_layer_size]), name='second_hidden_weights')
  second_hidden_biases = tf.Variable(
    tf.zeros([second_hidden_layer_size]), name='second_hidden_biases')
  second_hidden_layer = tf.nn.relu(tf.matmul(first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  
  # Build the output layer.
  output_weights = tf.Variable(tf.truncated_normal([second_hidden_layer_size, 3]), name='output_weights')
  output_biases = tf.Variable(tf.zeros([3]), name='output_biases')
  output_logits = tf.matmul(second_hidden_layer, output_weights) + output_biases
  training_estimate = tf.nn.softmax(output_logits)

  # Calculate loss and setup the optimizer.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_logits, tf_training_labels))
  l2_regularization = (tf.nn.l2_loss(output_weights) +
                       tf.nn.l2_loss(first_hidden_weights) +
                       tf.nn.l2_loss(second_hidden_weights))
  loss += regularization_factor * l2_regularization
  training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

  # Setup validation.  We have to reshape into a "dense tensor"
  # by, essentially, combining this array of arrays into a true matrix.
  tf_validation_pixel_data = tf.constant(
    validation_pixel_data.reshape((-1, card_width*card_height*card_channels)).astype(np.float32))
  validation_first_hidden_layer = tf.nn.relu(
    tf.matmul(tf_validation_pixel_data, first_hidden_weights) + first_hidden_biases)
  validation_second_hidden_layer = tf.nn.relu(
    tf.matmul(validation_first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  validation_logits = tf.matmul(validation_second_hidden_layer, output_weights) + output_biases
  validation_estimate = tf.nn.softmax(validation_logits)

  # Setup the final test run.
  tf_test_pixel_data = tf.constant(
    test_pixel_data.reshape((-1, card_width*card_height*card_channels)).astype(np.float32))
  test_first_hidden_layer = tf.nn.relu(
    tf.matmul(tf_test_pixel_data, first_hidden_weights) + first_hidden_biases)
  test_second_hidden_layer = tf.nn.relu(
    tf.matmul(test_first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  test_logits = tf.matmul(test_second_hidden_layer, output_weights) + output_biases
  test_estimate = tf.nn.softmax(test_logits)